In [ ]:
!pip install tensorflow matplotlib opencv-python


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Activation, GlobalAveragePooling2D, Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle
import cv2
import glob

In [2]:
# Set global variables
global inputShape, size
size = 100  # Define the input image size

In [10]:
# Define the CNN model
def kerasModel4():
    model = Sequential()
    model.add(Conv2D(16, (8, 8), strides=(4, 4), padding='valid', input_shape=(size, size, 1)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (5, 5), padding="same"))
    model.add(Activation('relu'))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(512))
    model.add(Dropout(0.1))
    model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    return model


In [11]:
# Paths to datasets (update these paths)
from google.colab import drive
drive.mount('/content/drive')  # Mount Google Drive if datasets are stored there


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
pothole_train_path = '/content/drive/MyDrive/Colab Notebooks/withpotholes/*.jpg'
nonpothole_train_path = '/content/drive/MyDrive/Colab Notebooks/withoutpotholes/*.jpg'
pothole_test_path = '/content/drive/MyDrive/Colab Notebooks/pot/*.jpg'
nonpothole_test_path = '/content/drive/MyDrive/Colab Notebooks/plain/*.jpg'


# New Section

In [55]:
# Load Training Data: Potholes
pothole_train_images = glob.glob(pothole_train_path)
train1 = [cv2.imread(img, 0) for img in pothole_train_images]
train1 = [cv2.resize(img, (size, size)) for img in train1]
temp1 = np.asarray(train1)

In [58]:
for img_path in nonpothole_train_images[:5]:  # Test with a few images
    img = cv2.imread(img_path, 0)
    if img is None:
        print(f"Failed to load image: {img_path}")
    else:
        print(f"Successfully loaded: {img_path}")

Successfully loaded: /content/drive/MyDrive/Colab Notebooks/withoutpotholes/1.Longeststraight0.jpg
Successfully loaded: /content/drive/MyDrive/Colab Notebooks/withoutpotholes/10.island-road-trip-smooth-roads-road-trip-sunny-day-sky-having-cumulus-nimbus-clouds-along-empty-smooth-black-124987092.jpg
Successfully loaded: /content/drive/MyDrive/Colab Notebooks/withoutpotholes/102.Two-new-phases-of-the-Dubai-Expo-2020--033_16ce7757dde_large.jpg
Successfully loaded: /content/drive/MyDrive/Colab Notebooks/withoutpotholes/104.nhai-kmOB--621x414@LiveMint_1562329017788.jpg
Successfully loaded: /content/drive/MyDrive/Colab Notebooks/withoutpotholes/100.5b23afff15e9f96fe6738e67.jpg


In [60]:
# Load Training Data: Non-Potholes
nonpothole_train_images = glob.glob(nonpothole_train_path)
train2 = []

for img_path in nonpothole_train_images:
    img = cv2.imread(img_path, 0)
    if img is not None:  # Check if the image is loaded correctly
        resized_img = cv2.resize(img, (size, size))
        train2.append(resized_img)
    else:
        print(f"Skipping invalid image: {img_path}")

temp2 = np.asarray(train2)

print(f"Number of non-pothole images loaded: {len(train2)}")
print(f"Shape of temp2: {temp2.shape}")


Skipping invalid image: /content/drive/MyDrive/Colab Notebooks/withoutpotholes/340.imagef7973d0b-ad11-4bbc-bc81-5d606377259f.jpg
Number of non-pothole images loaded: 336
Shape of temp2: (336, 100, 100)


In [61]:
# Load Testing Data: Potholes
pothole_test_images = glob.glob(pothole_test_path)
test1 = [cv2.imread(img, 0) for img in pothole_test_images]
test1 = [cv2.resize(img, (size, size)) for img in test1]
temp3 = np.asarray(test1)


In [62]:
# Load Testing Data: Non-Potholes
nonpothole_test_images = glob.glob(nonpothole_test_path)
test2 = [cv2.imread(img, 0) for img in nonpothole_test_images]
test2 = [cv2.resize(img, (size, size)) for img in test2]
temp4 = np.asarray(test2)

In [63]:
# Combine and shuffle training data
X_train = np.concatenate((temp1, temp2), axis=0)
X_test = np.concatenate((temp3, temp4), axis=0)

In [64]:

y_train1 = np.ones(len(temp1), dtype=int)
y_train2 = np.zeros(len(temp2), dtype=int)
y_test1 = np.ones(len(temp3), dtype=int)
y_test2 = np.zeros(len(temp4), dtype=int)

In [65]:

y_train = np.concatenate((y_train1, y_train2), axis=0)
y_test = np.concatenate((y_test1, y_test2), axis=0)

In [66]:
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

In [67]:
# Reshape and preprocess data
X_train = X_train.reshape(X_train.shape[0], size, size, 1) / 255.0
X_test = X_test.reshape(X_test.shape[0], size, size, 1) / 255.0

In [68]:

y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)


In [70]:
# Build and compile the model
inputShape = (size, size, 1)
model = kerasModel4()
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [71]:
print(f"Number of Pothole Training Images: {len(pothole_train_images)}")
print(f"Number of Non-Pothole Training Images: {len(nonpothole_train_images)}")
print(f"Number of Pothole Testing Images: {len(pothole_test_images)}")
print(f"Number of Non-Pothole Testing Images: {len(nonpothole_test_images)}")


Number of Pothole Training Images: 316
Number of Non-Pothole Training Images: 337
Number of Pothole Testing Images: 9
Number of Non-Pothole Testing Images: 8


In [72]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)


Shape of X_train: (652, 100, 100, 1)
Shape of y_train: (652, 2)


In [73]:
y_train1 = np.ones(len(temp1), dtype=int)
y_train2 = np.zeros(len(temp2), dtype=int)
y_test1 = np.ones(len(temp3), dtype=int)
y_test2 = np.zeros(len(temp4), dtype=int)

In [74]:
y_train = np.concatenate((y_train1, y_train2), axis=0)
y_test = np.concatenate((y_test1, y_test2), axis=0)


In [75]:
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

In [76]:
X_train = X_train.reshape(X_train.shape[0], size, size, 1) / 255.0
X_test = X_test.reshape(X_test.shape[0], size, size, 1) / 255.0

In [77]:
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [78]:
inputShape = (size, size, 1)
model = kerasModel4()
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [82]:

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_split=0.1)

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.5478 - loss: 0.6912 - val_accuracy: 0.5303 - val_loss: 0.6916
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.4941 - loss: 0.6942 - val_accuracy: 0.5303 - val_loss: 0.6920
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.4947 - loss: 0.6937 - val_accuracy: 0.5303 - val_loss: 0.6920
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.5158 - loss: 0.6927 - val_accuracy: 0.5303 - val_loss: 0.6919
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - accuracy: 0.5049 - loss: 0.6933 - val_accuracy: 0.5303 - val_loss: 0.6919
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.4922 - loss: 0.6940 - val_accuracy: 0.5303 - val_loss: 0.6919
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - accuracy: 0.5232 - loss: 0.6924 - val_accuracy: 0.5303 - val_loss: 0.6917
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.5270 - loss: 0.6920 - val_accuracy: 0

In [80]:

# Evaluate the model
metrics = model.evaluate(X_test, y_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print(f'{metric_name}: {metric_value}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4706 - loss: 0.6951
loss: 0.6950876116752625
compile_metrics: 0.47058823704719543


In [81]:
# Save the model
model.save('pothole_model.h5')
print("Model saved as 'pothole_model.h5'")

Model saved as 'pothole_model.h5'


In [83]:
print("Training class distribution:")
print(np.sum(y_train, axis=0))  # Sum per class

print("Test class distribution:")
print(np.sum(y_test, axis=0))


Training class distribution:
[336. 316.]
Test class distribution:
[8. 9.]


In [84]:
import pickle

# Save the model file (.h5) using pickle
with open('pothole_model.pkl', 'wb') as f:
    pickle.dump('pothole_model.h5', f)

print("Model saved as 'pothole_model.pkl'")


Model saved as 'pothole_model.pkl'


In [85]:
from google.colab import files

# Download the pickle file to local machine
files.download('pothole_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>